Transfer Learning Xception

#  Sexual Harassment Detection
Sexual Harassment Detection Using a Transfer Learning method Xception

## Importing necessary modules
### Data Processing and Manipulation
cv2 : Used for image processing tasks<br>
glob : Used to find file paths matching a pattern<br>
os : Used for interacting with the operating system<br>
numpy : Used for numerical computations and array handling<br>
pandas : Used for data manipulation and analysis<br>
### For Visualization
matplotlib.pyplot : Used for plotting graphs and images<br>
tqdm : Used to display progress bars during iterations<br>
### For Building ML model
keras : High-level neural networks API for building and training models<br>
tensorflow : Deep Learning framework used as backend for Keras<br>
sklearn : Library for machine learning tasks, such as model evaluation


In [ ]:
import tensorflow as tf
import os
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline

if not os.path.isdir('models'):
    os.mkdir('models')
    
print('TensorFlow version:', tf.__version__)
print('Is using GPU?', tf.test.is_gpu_available())

In [2]:
import cv2
from glob import glob
from scipy import stats as s
import keras
from keras.models import Sequential
import tensorflow.keras.applications.xception
from keras.applications.xception import Xception
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.layers import Flatten

### Reading a CSV file
This csv file contains two columns which has the location of the images in our dataset and the class label (0-Healthy Environment / 1-Harassment Detected) associated with it.

The csv file - 'harassment_as_csv.csv' is generated as an output of 'preprocessing.ipynb'
#### Loading 
Reads the csv file using pandas and stores it in the 'train' DataFrame.

In [8]:
train = pd.read_csv(r"D:\harrasment_as_csv.csv")
train.head()

,image,class
0,images/harassment_yes\Screenshot 2023-07-22 01...,1
1,images/harassment_yes\Screenshot 2023-07-22 01...,1
2,images/harassment_yes\Screenshot 2023-07-22 01...,1
3,images/harassment_yes\Screenshot 2023-07-22 01...,1
4,images/harassment_yes\Screenshot 2023-07-22 01...,1


## Preprocessing Images 
This iterates through each row of the 'train' DataFrame, loads the image specified by the 'image' column.
#### Preprocessing Steps:
The image is resized to (224,224,3)

Converts it to a NumPy array.

Normalizes the pixel values to the range [0,1]

These processed images are then stored in the 'train_image' list.
This list is then converted to a NumPy array 'X'

In [53]:
# Import the load_img function from tensorflow.keras
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [11]:
# creating an empty list
train_image = []

# for loop to read and store frames
for i in tqdm(range(train.shape[0])):
    # loading the image and keeping the target size as (224, 224)
    img = load_img(train['image'][i], target_size=(224,224))
    # converting it to array
    img = img_to_array(img)
    # normalizing the pixel value
    img = img/255.0
    # appending the image to the train_image list
    train_image.append(img)

# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 92.01it/s]


(1000, 224, 224, 3)

#### Separating Target
Extracts the 'class' column from the 'train' DataFrame, which contains the class labels of the images.

#### Dataset Segregation
The train_test_split function splits the 'x' and 'y' into training and testing parts.
The split is done with test size of 20% .

The 'stratify' parameter ensures that the class distribution is preserved in both the sets.

In [18]:
# separating the target
y = train['class']

# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

The function 'get_dummies' converts the categorical class labels into one-hot enocoded format.

In [19]:
# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

## Loading our Pre-trained Model
A pre-trained VGG16 model is loaded from keras with pre-trained weights on the ImageNet dataset.
The fully connected layers at the top of the network is excluded by setting the parameter 'include_top' to False

In [22]:
# creating the base model of pre-trained Xception model
base_model = Xception(weights='imagenet', include_top=False, input_shape=(224,224,3))

### Preprocessing
preprocessing the training and validation images

In [25]:
# Preprocess the training and validation images
X_train_preprocessed = keras.applications.xception.preprocess_input(X_train)
X_test_preprocessed = keras.applications.xception.preprocess_input(X_test)

### Extracting Features
Here we extract features from the images in the training and testing sets using our base_model - Xception.
The 'predict' method is applied to both sets to obtain feature vectors for each image.

In [26]:
# extracting features for training frames
X_train_features = base_model.predict(X_train_preprocessed)

# extracting features for validation frames
X_test_features = base_model.predict(X_test_preprocessed)

# flatten the feature arrays
X_train_flatten = X_train_features.reshape(X_train_features.shape[0], -1)
X_test_flatten = X_test_features.reshape(X_test_features.shape[0], -1)

print("Shape of X_train_flatten:", X_train_flatten.shape)
print("Shape of X_test_flatten:", X_test_flatten.shape)

7/7 [==============================] - 6s 870ms/step
Shape of X_train_flatten: (800, 100352)
Shape of X_test_flatten: (200, 100352)


In [27]:
# extracting features for training frames
X_train = base_model.predict(X_train)
X_train.shape

25/25 [==============================] - 25s 985ms/step


(800, 7, 7, 2048)

In [28]:
# extracting features for validation frames
X_test = base_model.predict(X_test)
X_test.shape

7/7 [==============================] - 6s 821ms/step


(200, 7, 7, 2048)

The extracted features are reshaped into a format suitable for feeding into the subsequent fully connected layers. This depends on the architecture of the VGG16 model.

In [30]:
# reshaping the training as well as validation frames in single dimension
X_train = X_train.reshape(800,7*7*2048)
X_test = X_test.reshape(200,7*7*2048)

The feature vectors are normalized by dividing them by the maximum value of the training set.

In [31]:
# normalizing the pixel values
max = X_train.max()
X_train = X_train/max
X_test = X_test/max
X_train.shape

(800, 100352)

## Model Architecture
Fully Connected(Dense) layers with ReLU activation function.

Dropout Layers for regularization.

Dense Layer with softmax activation for classification.

In [36]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(7*7*2048,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

#### Callback Definition 
A callback to save the model weights during training. It is saved in the 'weight.hdf5' file whenever the validation loss improves.

In [37]:
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

#### Model compilation
The model is compiled with a categorical cross-entropy loss function, Adam optimizer, and accuracy metric. 

In [38]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 1024)              102761472 
                                                                 
 dropout_4 (Dropout)         (None, 1024)              0         
                                                                 
 dense_6 (Dense)             (None, 512)               524800    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_7 (Dense)             (None, 256)               131328    
                                                                 
 dropout_6 (Dropout)         (None, 256)               0         
                                                                 
 dense_8 (Dense)             (None, 128)              

# Graphs,Plots and Visualization

## Sample Images
Plot of few sample images from the dataset along with their class labels

In [39]:
# training the model
model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=256)

Epoch 1/200
4/4 [==============================] - 6s 1s/step - loss: 0.7401 - accuracy: 0.4837 - val_loss: 0.6979 - val_accuracy: 0.5000
Epoch 2/200
4/4 [==============================] - 5s 1s/step - loss: 0.7754 - accuracy: 0.4837 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 3/200
4/4 [==============================] - 4s 958ms/step - loss: 0.7916 - accuracy: 0.4812 - val_loss: 0.6977 - val_accuracy: 0.5000
Epoch 4/200
4/4 [==============================] - 4s 968ms/step - loss: 0.7591 - accuracy: 0.4925 - val_loss: 0.6950 - val_accuracy: 0.5000
Epoch 5/200
4/4 [==============================] - 4s 953ms/step - loss: 0.7281 - accuracy: 0.4975 - val_loss: 0.6937 - val_accuracy: 0.5000
Epoch 6/200
4/4 [==============================] - 4s 952ms/step - loss: 0.7234 - accuracy: 0.4963 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 7/200
4/4 [==============================] - 4s 995ms/step - loss: 0.7145 - accuracy: 0.5100 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 8/200
4/4 [==

Epoch 59/200
4/4 [==============================] - 4s 941ms/step - loss: 0.6930 - accuracy: 0.5113 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 60/200
4/4 [==============================] - 4s 934ms/step - loss: 0.6932 - accuracy: 0.4913 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 61/200
4/4 [==============================] - 4s 948ms/step - loss: 0.6934 - accuracy: 0.4975 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 62/200
4/4 [==============================] - 4s 947ms/step - loss: 0.6936 - accuracy: 0.4875 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 63/200
4/4 [==============================] - 4s 940ms/step - loss: 0.6932 - accuracy: 0.5038 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 64/200
4/4 [==============================] - 4s 946ms/step - loss: 0.6934 - accuracy: 0.4900 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 65/200
4/4 [==============================] - 4s 919ms/step - loss: 0.6927 - accuracy: 0.5200 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 

4/4 [==============================] - 4s 957ms/step - loss: 0.6933 - accuracy: 0.4975 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 117/200
4/4 [==============================] - 4s 948ms/step - loss: 0.6933 - accuracy: 0.4988 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 118/200
4/4 [==============================] - 4s 948ms/step - loss: 0.6932 - accuracy: 0.4938 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 119/200
4/4 [==============================] - 4s 931ms/step - loss: 0.6933 - accuracy: 0.4875 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 120/200
4/4 [==============================] - 4s 943ms/step - loss: 0.6933 - accuracy: 0.4950 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 121/200
4/4 [==============================] - 4s 949ms/step - loss: 0.6932 - accuracy: 0.4975 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 122/200
4/4 [==============================] - 4s 979ms/step - loss: 0.6933 - accuracy: 0.4825 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 123/200

4/4 [==============================] - 4s 938ms/step - loss: 0.6933 - accuracy: 0.5000 - val_loss: 0.6934 - val_accuracy: 0.5000
Epoch 174/200
4/4 [==============================] - 4s 941ms/step - loss: 0.6935 - accuracy: 0.5000 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 175/200
4/4 [==============================] - 4s 974ms/step - loss: 0.6930 - accuracy: 0.5000 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 176/200
4/4 [==============================] - 4s 913ms/step - loss: 0.6935 - accuracy: 0.5000 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 177/200
4/4 [==============================] - 4s 926ms/step - loss: 0.6935 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 178/200
4/4 [==============================] - 4s 921ms/step - loss: 0.6929 - accuracy: 0.5088 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 179/200
4/4 [==============================] - 4s 921ms/step - loss: 0.6933 - accuracy: 0.4938 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 180/200

In [40]:
scores = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {scores[1]*100} ")

7/7 [==============================] - 0s 32ms/step - loss: 0.6932 - accuracy: 0.5000
Test Accuracy: 50.0 
